In [3]:
from selenium import webdriver
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [4]:
urls = []

for j in range(30):
    if j == 0:
        print('Crawling 시작')
    driver = webdriver.Chrome('chromedriver')
    url_page = str(j)
    driver.get(url='https://patents.google.com/?q=(solar+cell)&q=F24J2%2f14,F24J2%2f45,F24J2%2f46,F24J2%2f523&page='+url_page)
    driver.implicitly_wait(time_to_wait=5)
    info = driver.find_element_by_xpath('//*[@id="resultsContainer"]')
    contents = info.find_elements_by_tag_name('search-result-item')
    for i in range(len(contents)):
        address = contents[i].find_element_by_tag_name('state-modifier').get_attribute('data-result') # patent/name/en
        url = 'patents.google.com/%s' % address
        urls.append('http://'+url)
    if j == 35:
        print('Crawling 끝')

df = pd.DataFrame(urls,columns=['url'])
df.to_csv('url.csv')
print('crawling한 url주소를 url.csv 라는 이름으로 저장함')

Crawling 시작
crawling한 url주소를 url.csv 라는 이름으로 저장함


In [46]:
df_list = list(df['url'])
title = []
abstract = []
label_all = []
meta_label = []

for k in df_list:
#    print(k+' Crawling중입니다.')
    html = urlopen(k)
    bsObject = BeautifulSoup(html,'html.parser')
    
#    print('제목시작')
    for a in bsObject.find_all('title'):
        text = a.get_text()
        title.append(text)
#    print('제목 끝')
    
#    print('abstract 시작')
    for b in bsObject.find_all('meta',{'name':'description'}):
        b = str(b)
        b =b.replace('<meta content="\n','')
        c = b.replace('name="description"/>','')
        abstract.append(c)
#     print('abstract 끝')

#     print('ipc시작')
    label_fake = []   
    for a in bsObject.find_all('li',{'itemprop':'cpcs'}):
        label_all.append(a)
        label_fake.append(a)
#     print(len(label_fake),'개의 코드 찾음')
#     print("label_fake ipc 끝")
    
    o = []
    x = []

    for i in range(len(label_fake)):
        if str('<meta content="true" itemprop="Leaf"') in str(label_fake[i]):
            o.append(label_fake[i])
        else :
            x.append(label_fake[i])
            
#     print('o 에 대한거 완료')

    
    for j in range(len(o)):    
        meta = o[j].find_all('span',{'itemprop':'Code'})[0].get_text()
        meta_label.append(meta)
#    print('meta label 집어넣기 ')
    meta_label.append('--------------------')    

ml = ';'.join(meta_label)
final_ipc = ml.split('--------------------')

ipc_df = pd.DataFrame(final_ipc[:-1],columns=['Ipc'])
title_df = pd.DataFrame(title,columns=['Title'])
abstract_df = pd.DataFrame(abstract,columns=['Abstract'])

patent_crawled_df = pd.concat([title_df,abstract_df,ipc_df,],axis=1)

patent_crawled_df.to_csv("patent_crawling.csv")